# Imports

* Brings in **pandas/numpy** for data handling.
* **scikit-learn** modules for splitting, preprocessing, modeling, evaluation, calibration, and handling class imbalance.
    

In [8]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve, auc
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.utils.class_weight import compute_class_weight

In [9]:
import warnings
from sklearn.exceptions import UndefinedMetricWarning

# Suppress only UndefinedMetricWarning
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

# 1) Load data

In [10]:
df = pd.read_csv("hospital_readmission_dataset.csv")

# 2) Features and target


* **Features (X):** predictors (visits, age, conditions).
* **Target (y):** binary outcome (readmitted within 30 days).


In [11]:
features = ["prior_visits_last_year", "age", "chronic_conditions"]
X = df[features]
y = df["readmitted_30_days"]

# 3) Train/test split (stratified)


* Splits into **training (75%)** and **testing (25%)**.
* stratify=y ensures class balance across splits.

# 🔹 What stratify=y Does

* When you use train_test_split from **scikit-learn**, the parameter **stratify=y** ensures that the class distribution in your target variable (y) is preserved across the train and test sets.

* Without stratification: The split is random, so some classes might be overrepresented in the training set and underrepresented in the test set. This can cause **class imbalance** problems.

* With stratify=y: The function looks at the labels in y and makes sure that the **proportion of each class** in the train and test sets matches the proportion in the original dataset.


In [12]:

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# 4) Baseline: predict majority class

* Predicts all patients as not **readmitted** (majority class baseline).
* Useful for benchmarking: if your models don’t beat this, they’re not adding value.

In [13]:
baseline_pred = np.zeros_like(y_test)  # assume '0' is majority

In [14]:
print("Baseline classification report:")
print(classification_report(y_test, baseline_pred, digits=3))

Baseline classification report:
              precision    recall  f1-score   support

           0      0.784     1.000     0.879        98
           1      0.000     0.000     0.000        27

    accuracy                          0.784       125
   macro avg      0.392     0.500     0.439       125
weighted avg      0.615     0.784     0.689       125



# 5) Class weights for imbalance

* Handles **class imbalance** (e.g., fewer readmissions than non-readmissions).
* Assigns higher weight to minority class to avoid bias toward majority.

In [15]:
classes = np.unique(y)

In [16]:
class_weights = compute_class_weight(class_weight="balanced", classes=classes, y=y)

In [17]:
cw = {int(c): w for c, w in zip(classes, class_weights)}
print("Class weights:", cw)

Class weights: {0: np.float64(0.639386189258312), 1: np.float64(2.293577981651376)}


# 6) Interpretable model: Logistic Regression


* Pipeline standardizes features (important for regression).
* Logistic Regression is interpretable: coefficients show how each feature affects readmission odds.
* Evaluates with **classification report** and **ROC-AUC**.


In [18]:
logit_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(
        class_weight=cw, 
        max_iter=2000, 
        solver="lbfgs", 
        C=1.0
    ))
])

In [19]:
logit_pipe.fit(X_train, y_train)
logit_proba = logit_pipe.predict_proba(X_test)[:, 1]
logit_pred = (logit_proba >= 0.5).astype(int)

In [20]:
print("\nLogistic Regression classification report:")
print(classification_report(y_test, logit_pred, digits=3))
print("Logistic ROC-AUC:", roc_auc_score(y_test, logit_proba))


Logistic Regression classification report:
              precision    recall  f1-score   support

           0      0.873     0.704     0.780        98
           1      0.370     0.630     0.466        27

    accuracy                          0.688       125
   macro avg      0.621     0.667     0.623       125
weighted avg      0.765     0.688     0.712       125

Logistic ROC-AUC: 0.705026455026455


# 7. Threshold Optimization

* Instead of default 0.5, finds the threshold that maximizes **F1 score** (balance between precision and recall).
* This is critical in healthcare where false negatives (missed readmissions) can be costly.

In [21]:
prec, rec, th = precision_recall_curve(y_test, logit_proba)
f1 = 2 * (prec * rec) / (prec + rec + 1e-9)
best_idx = np.nanargmax(f1)
best_threshold = th[best_idx] if best_idx < len(th) else 0.5

In [22]:
# Optional: choose threshold by maximizing F1 or Youden’s J on validation
print("Best threshold (by F1):", round(float(best_threshold), 3))

Best threshold (by F1): 0.568


# 8. Gradient Boosting (Stronger Model)

* Gradient Boosting captures nonlinear relationships and interactions.
* Calibration ensures predicted probabilities are reliable (important for risk scores).
* Evaluates with classification report and ROC-AUC.

In [23]:
gb = GradientBoostingClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=3,
    random_state=42
)
gb_cal = CalibratedClassifierCV(gb, cv=5, method="isotonic")
gb_cal.fit(X_train, y_train)
gb_proba = gb_cal.predict_proba(X_test)[:, 1]
gb_pred = (gb_proba >= best_threshold).astype(int)

print("\nGradient Boosting classification report (using best threshold):")
print(classification_report(y_test, gb_pred, digits=3))
print("Gradient Boosting ROC-AUC:", roc_auc_score(y_test, gb_proba))


Gradient Boosting classification report (using best threshold):
              precision    recall  f1-score   support

           0      0.798     0.969     0.876        98
           1      0.500     0.111     0.182        27

    accuracy                          0.784       125
   macro avg      0.649     0.540     0.529       125
weighted avg      0.734     0.784     0.726       125

Gradient Boosting ROC-AUC: 0.6077097505668935


# 9) Feature importance (GBM)

* Shows which features contribute most to predictions.
* Helps clinicians understand drivers of readmission risk.

In [24]:
importances = gb.feature_importances_ if hasattr(gb, "feature_importances_") else None
if importances is not None:
    fi = pd.Series(importances, index=features).sort_values(ascending=False)
    print("\nGradient Boosting feature importances:")
    print(fi)

# 10) Produce patient-level risk scores and predictions for the full dataset


* Produces **risk scores** (probabilities) and **binary predictions** for each patient.
* Useful for clinical decision support (flagging high-risk patients).


In [25]:
df["risk_logit"] = logit_pipe.predict_proba(X)[:, 1]
df["risk_gb"] = gb_cal.predict_proba(X)[:, 1]
df["pred_logit"] = (df["risk_logit"] >= best_threshold).astype(int)
df["pred_gb"] = (df["risk_gb"] >= best_threshold).astype(int)

# 11) Save outputs

* Exports patient-level predictions for further analysis or reporting.

In [26]:
df_out = df[["patient_id"] + features + ["readmitted_30_days", "risk_logit", "risk_gb", "pred_logit", "pred_gb"]]
df_out.to_csv("readmission_predictions.csv", index=False)
print("\nSaved: readmission_predictions.csv")


Saved: readmission_predictions.csv


# 🚀 Key Strengths of Your Workflow

* **Baseline comparison** ensures models are meaningful.
* **Class imbalance handling** improves fairness.
* **Threshold tuning** aligns with healthcare priorities.
* **Interpretable + strong models** balance transparency and accuracy.
* **Patient-level outputs** make results actionable.
    